# Import Libraries

In [119]:
import pandas as pd

# Obtain Data

Read csv file with consumer reports ratings into pandas df

In [147]:
cr_df = pd.read_csv('Data/Ratings.csv')
cr_df = cr_df.iloc[:, range(1, len(cr_df.columns), 2)]
cr_df.columns = [x.replace('.1', '') for x in cr_df.columns]
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('mercedes-benz', 'mercedes benz'))
cr_df['Model'] = cr_df['Model'].apply(lambda x : x.replace('land-rover', 'land rover'))
cr_df['Brand'] = cr_df['Model'].apply(lambda x : x.split('-')[0])
def model_name(row):
    return row['Model'].replace(row['Brand']+'-', '')
cr_df['Model'] = cr_df.apply(model_name, axis=1)
print('cr_df shape: ', cr_df.shape)
cr_df.head()

cr_df shape:  (295, 18)


,Model,Reliability,Acceleration,Braking,Ride,Noise,Front Seat Comfort,Rear Seat Comfort,Interior Fit and Finish,Trunk/Cargo Area,Fuel Economy,Routine Handling,Headlights,Driving Position,Third Seat Comfort,Usability,Max. Load,Brand
0,v60,1 / 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,volvo
1,500l,NaN,3 / 5,4 / 5,2 / 5,3 / 5,3 / 5,4 / 5,3 / 5,2 / 5,4 / 5,4 / 5,3 / 5,3 / 5,NaN,3 / 5,860 lb.,fiat
2,xf,NaN,5 / 5,5 / 5,5 / 5,4 / 5,5 / 5,4 / 5,5 / 5,2 / 5,3 / 5,5 / 5,3 / 5,4 / 5,NaN,3 / 5,960 lb.,jaguar
3,tt,NaN,5 / 5,5 / 5,3 / 5,3 / 5,5 / 5,1 / 5,5 / 5,2 / 5,4 / 5,5 / 5,2 / 5,4 / 5,NaN,2 / 5,770 lb.,audi
4,mirai,NaN,3 / 5,3 / 5,4 / 5,4 / 5,4 / 5,3 / 5,4 / 5,2 / 5,5 / 5,2 / 5,4 / 5,3 / 5,NaN,3 / 5,690 lb.,toyota


Read csv file with auto sales data into pandas df

In [149]:
sales_df = pd.read_excel('Data/Auto_Sales/1-19-auto-sales.xls', header=4)
sales_df.rename(columns={'Unnamed: 0': 'Model'}, inplace=True)
sales_df.dropna(inplace=True)
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("(I)", ""))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("*", ""))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.replace("Total", ""))
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.strip())
sales_df.drop(columns=['change', 2018], inplace=True)
sales_df = sales_df.groupby('Model').agg({'Model':'first', 2019:'max'})
sales_df.reset_index(inplace=True, drop=True)
sales_df['Model'] = sales_df['Model'].apply(lambda x: x.lower())
print('sales_df shape: ', sales_df.shape)
sales_df.head()

sales_df shape:  (537, 2)


,Model,2019
0,2 series,578.0
1,200,12.0
2,3 series,1226.0
3,300,2078.0
4,370z,173.0


In [155]:
merged_df = pd.merge(sales_df, cr_df, on='Model')
merged_df

,Model,2019,Reliability,Acceleration,Braking,Ride,Noise,Front Seat Comfort,Rear Seat Comfort,Interior Fit and Finish,Trunk/Cargo Area,Fuel Economy,Routine Handling,Headlights,Driving Position,Third Seat Comfort,Usability,Max. Load,Brand
0,300,2078.0,5 / 5,4 / 5,4 / 5,4 / 5,5 / 5,5 / 5,5 / 5,5 / 5,3 / 5,3 / 5,4 / 5,1 / 5,4 / 5,NaN,4 / 5,865 lb.,chrysler
1,4runner,10335.0,4 / 5,4 / 5,4 / 5,2 / 5,4 / 5,4 / 5,4 / 5,3 / 5,4 / 5,2 / 5,2 / 5,4 / 5,3 / 5,NaN,4 / 5,1155 lb.,toyota
2,500,261.0,NaN,4 / 5,5 / 5,1 / 5,2 / 5,3 / 5,1 / 5,3 / 5,1 / 5,4 / 5,5 / 5,4 / 5,3 / 5,NaN,3 / 5,700 lb.,fiat
3,500l,56.0,NaN,3 / 5,4 / 5,2 / 5,3 / 5,3 / 5,4 / 5,3 / 5,2 / 5,4 / 5,4 / 5,3 / 5,3 / 5,NaN,3 / 5,860 lb.,fiat
4,500x,274.0,NaN,3 / 5,4 / 5,2 / 5,2 / 5,3 / 5,3 / 5,3 / 5,1 / 5,3 / 5,3 / 5,2 / 5,3 / 5,NaN,4 / 5,1080 lb.,fiat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,yaris,1323.0,NaN,3 / 5,3 / 5,3 / 5,3 / 5,3 / 5,2 / 5,3 / 5,3 / 5,5 / 5,5 / 5,3 / 5,3 / 5,NaN,3 / 5,850 lb.,toyota
196,yukon,3644.0,1 / 5,4 / 5,4 / 5,3 / 5,5 / 5,4 / 5,4 / 5,4 / 5,4 / 5,1 / 5,2 / 5,4 / 5,4 / 5,1 / 5,5 / 5,1580 lb.,gmc
197,z4,0.0,NaN,5 / 5,5 / 5,1 / 5,2 / 5,5 / 5,NaN,5 / 5,2 / 5,4 / 5,5 / 5,2 / 5,3 / 5,NaN,3 / 5,465 lb.,bmw
198,i3,255.0,NaN,4 / 5,4 / 5,4 / 5,5 / 5,4 / 5,3 / 5,4 / 5,2 / 5,5 / 5,4 / 5,2 / 5,3 / 5,NaN,3 / 5,650 lb.,bmw


In [159]:
test = pd.merge(sales_df, cr_df, on='Model', how='right')
test[test[2019].isna()]['Model'].values

array(['v60', '6', '1500-classic', 'gle-coupe', 'a8', 'range-rover-sport',
       'range-rover', 'discovery-sport', '8-series', 'silverado-3500hd',
       'f-250', 'silverado-1500', 'versa-note', 'levante', 'f-350',
       'cooper', 'sierra-2500hd', 'prius-prime', 'a4', 'sprinter',
       'corolla-hatchback', 'rx-l', '3-series', '718-cayman',
       'eclipse-cross', 'romeo-giulia', 'model-3', 'a5', 'z',
       'land-cruiser', 'santa-fe', '86', 'golf-alltrack',
       'golf-sportwagen', 'model-s', 'model-x', 'grand-caravan',
       'yukon-xl', 'x7', '3', 'a6', 's90', 'arteon', 'e-tron', 'rlx',
       'range-rover-velar', '5-series', 'a-class', 'gle', 'allroad',
       'slc', '4-series', 's4', 'kona-electric', 'sierra-1500', 'a7',
       '6-series', 's-class', 'prius-c', 'silverado-2500hd', 'c-class',
       'ghibli', 'mx-5-miata', 'grand-cherokee', 'gti', 'nautilus', 'glc',
       'sierra-3500hd', 'rogue-sport', '3500', 'romeo-4c', 'golf',
       'outlander-sport', 'metris', '1500', 'e-

In [160]:
test[test[2019].notna()]['Model'].values

array(['500l', 'xf', 'tt', 'mirai', 'encore', 'a3', 'sl', 'envision',
       'ls', 'yaris', 'q5', 'x2', 'fortwo', 'tacoma', 'gs', 'xj', 'fit',
       'sonic', 'accent', 'legacy', 'qx80', 'terrain', 'i-pace', 'gla',
       '500', 'insight', 'q50', 'mustang', 'tundra', 'xc90', 'q3', 'kona',
       'q7', 'titan', 'optima', 'qx60', 'regal', 'spark', 'lacrosse',
       'panamera', 'ridgeline', 'forte', 'expedition', 'yukon', 'nsx',
       'mdx', 'escalade', 'cherokee', 'outlander', 'fiesta', 'xt5', 'i3',
       'impreza', 'explorer', 'pathfinder', 'armada', 'qx50', 'cx-9',
       'x4', 'tlx', 'challenger', 'nx', 'i8', 'volt', 'xts', '911',
       'ioniq', 'macan', 'versa', 'f-pace', 'cla', 'avalon', 'sentra',
       'cascada', 'outback', 'impala', 'sequoia', '500x', 'flex', 'cruze',
       'passat', 'continental', 'bolt', 'x3', 'corvette', 'elantra',
       'ilx', 'q8', 'lx', 'beetle', 'q70', 'xe', 'edge', 'camaro', 'c-hr',
       'forester', 'fusion', 'camry', 'canyon', 'sedona', 'r8', 'ct

In [161]:
sales_df['Model'].values

array(['2 series', '200', '3 series', '300', '370z', '4 series', '4c',
       '4runner', '5 series', '500', '500l', '500x', '6 series',
       '60 series', '7 series', '718', '8 series', '86 (incl. fr-s)',
       '90 series', '911', 'a3', 'a4/s4', 'a5/s5', 'a6/s6', 'a7/s7',
       'a8/s8', 'american honda', 'aston martin car', 'ats', 'acadia',
       'accent', 'accord', 'accord (d)', 'acura', 'acura (d)',
       'acura car', 'acura car (d)', 'acura truck', 'alfa romeo',
       'alfa romeo car', 'alfa romeo truck', 'altima', 'american honda',
       'american honda (d)', 'american honda car', 'american honda truck',
       'armada', 'ascent', 'atlas', 'audi', 'audi car', 'audi truck',
       'avalon', 'avenger', 'azera', 'b class', 'bmw', 'bmw (d)',
       'bmw of n.a.', 'bmw car', 'bmw of n.a.', 'bmw of n.a. (d)',
       'bmw of n.a. car', 'bmw of n.a. truck', 'bmw truck',
       'bmw truck (d)', 'brz', 'beetle', 'bentley', 'bentley car',
       'bentley truck', 'blazer', 'bolt', 'buic

In [162]:
cr_df['Model'].values

array(['v60', '500l', 'xf', 'tt', 'mirai', '6', 'encore', 'a3',
       '1500-classic', 'sl', 'envision', 'gle-coupe', 'a8', 'ls',
       'range-rover-sport', 'range-rover', 'discovery-sport', 'yaris',
       'q5', 'x2', 'fortwo', 'tacoma', '8-series', 'gs',
       'silverado-3500hd', 'f-250', 'xj', 'silverado-1500', 'versa-note',
       'fit', 'sonic', 'levante', 'accent', 'f-350', 'legacy', 'cooper',
       'qx80', 'terrain', 'i-pace', 'gla', '500', 'insight',
       'sierra-2500hd', 'q50', 'mustang', 'tundra', 'xc90', 'q3', 'kona',
       'prius-prime', 'a4', 'q7', 'sprinter', 'titan', 'optima',
       'corolla-hatchback', 'rx-l', 'qx60', 'regal', '3-series',
       '718-cayman', 'eclipse-cross', 'romeo-giulia', 'spark', 'model-3',
       'lacrosse', 'panamera', 'ridgeline', 'a5', 'z', 'forte',
       'land-cruiser', 'expedition', 'yukon', 'nsx', 'mdx', 'santa-fe',
       'escalade', 'cherokee', '86', 'golf-alltrack', 'outlander',
       'fiesta', 'xt5', 'i3', 'golf-sportwagen', 'mod

In [139]:
sales_df[sales_df['Model']=='yukon']

,Model,2019
530,yukon,3644.0


In [144]:
cr_df.iloc[0]['Brand']

'volvo'

In [164]:
cr_df[cr_df['Brand']=='volvo']

,Model,Reliability,Acceleration,Braking,Ride,Noise,Front Seat Comfort,Rear Seat Comfort,Interior Fit and Finish,Trunk/Cargo Area,Fuel Economy,Routine Handling,Headlights,Driving Position,Third Seat Comfort,Usability,Max. Load,Brand
0,v60,1 / 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,volvo
46,xc90,1 / 5,4 / 5,5 / 5,3 / 5,4 / 5,5 / 5,5 / 5,5 / 5,3 / 5,2 / 5,3 / 5,3 / 5,5 / 5,1 / 5,2 / 5,1210 lb.,volvo
110,s90,NaN,4 / 5,4 / 5,3 / 5,4 / 5,5 / 5,4 / 5,5 / 5,3 / 5,3 / 5,4 / 5,3 / 5,4 / 5,NaN,2 / 5,950 lb.,volvo
177,xc60,4 / 5,4 / 5,5 / 5,3 / 5,4 / 5,5 / 5,4 / 5,5 / 5,3 / 5,3 / 5,4 / 5,4 / 5,4 / 5,NaN,2 / 5,950 lb.,volvo
225,xc40,3 / 5,4 / 5,4 / 5,3 / 5,4 / 5,5 / 5,4 / 5,4 / 5,2 / 5,3 / 5,4 / 5,4 / 5,4 / 5,NaN,1 / 5,925 lb.,volvo
247,s60,1 / 5,4 / 5,5 / 5,3 / 5,4 / 5,4 / 5,3 / 5,4 / 5,2 / 5,4 / 5,4 / 5,3 / 5,4 / 5,NaN,2 / 5,890 lb.,volvo
